In [6]:
import requests
import datetime

from heatmap.definitions.timeseries import PowerCarbonIntensity
from heatmap.definitions.ng_carbon_intensity import Response as NgCarbonIntensityResponse

In [2]:
start_date = datetime.datetime(2024, 1, 1, 0, 0)
end_date = datetime.datetime(2024, 1, 1, 12, 59)

r = requests.get(f'https://api.carbonintensity.org.uk/intensity/{start_date.strftime("%Y-%m-%dT%H:%MZ")}/{end_date.strftime("%Y-%m-%dT%H:%MZ")}', headers={'Accept': 'application/json'})
 
ng_carbon_intensity_response = NgCarbonIntensityResponse.model_validate(r.json())

ng_carbon_intensity_response

Response(data=[Data(from_=datetime.datetime(2023, 12, 31, 23, 30, tzinfo=TzInfo(UTC)), to=datetime.datetime(2024, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), intensity=Intensity(forecast=68.0, actual=63.0, index='low')), Data(from_=datetime.datetime(2024, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), to=datetime.datetime(2024, 1, 1, 0, 30, tzinfo=TzInfo(UTC)), intensity=Intensity(forecast=64.0, actual=64.0, index='low')), Data(from_=datetime.datetime(2024, 1, 1, 0, 30, tzinfo=TzInfo(UTC)), to=datetime.datetime(2024, 1, 1, 1, 0, tzinfo=TzInfo(UTC)), intensity=Intensity(forecast=62.0, actual=64.0, index='low')), Data(from_=datetime.datetime(2024, 1, 1, 1, 0, tzinfo=TzInfo(UTC)), to=datetime.datetime(2024, 1, 1, 1, 30, tzinfo=TzInfo(UTC)), intensity=Intensity(forecast=65.0, actual=62.0, index='low')), Data(from_=datetime.datetime(2024, 1, 1, 1, 30, tzinfo=TzInfo(UTC)), to=datetime.datetime(2024, 1, 1, 2, 0, tzinfo=TzInfo(UTC)), intensity=Intensity(forecast=61.0, actual=71.0, index='low')), Data(from_=datetime.da

In [9]:
power_carbon_intensities = [
    PowerCarbonIntensity(
            start_time=data_record.from_,
            end_time=data_record.to,
            value=data_record.intensity.actual
        )
    for data_record 
    in ng_carbon_intensity_response.data
    if data_record.intensity.actual is not None
]

power_carbon_intensities

[PowerCarbonIntensity(start_time=datetime.datetime(2023, 12, 31, 23, 30, tzinfo=TzInfo(UTC)), value=63.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), value=64.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 0, 30, tzinfo=TzInfo(UTC)), value=64.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 1, 0, tzinfo=TzInfo(UTC)), value=62.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 1, 30, tzinfo=TzInfo(UTC)), value=71.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 2, 0, tzinfo=TzInfo(UTC)), value=66.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 2, 30, tzinfo=TzInfo(UTC)), value=67.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 3, 0, tzinfo=TzInfo(UTC)), value=64.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 3, 30, tzinfo=TzInfo(UTC)), value=60.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2024, 1, 1, 4,